In [15]:
import numpy as np
import PIL
import sys, os
from sklearn.decomposition import sparse_encode
from sklearn.feature_extraction.image import extract_patches_2d
numInputs = 192
batchSize = 16
IMAGE_SIZE = (32,32,3)
IMAGE_FILE = "cifar-10/test_batch"

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def fullImage2Patches(image):
    # Make batch of random image
    patches = np.zeros((numInputs, batchSize))  # getting sequential 8x8 RGB patches (192 pixels each)
    red = np.reshape(image[0:1024],(32,32))
    green = np.reshape(image[1024:2048],(32,32))
    blue = np.reshape(image[2048:3072],(32,32))

    for i in range(4):
        for j in range(4):
            red_layer = red[0 +8 *i:8 + 8*i,0 + 8*j:8 + 8*j].flatten()
            green_layer = green[0 + 8*i:8 + 8*i, 0 + 8*j:8 + 8*j].flatten()
            blue_layer = blue[0 + 8*i:8 + 8*i, 0 + 8*j:8 + 8*j].flatten()
            patches[:,i*4+j] = np.concatenate((red_layer,green_layer,blue_layer))


    return patches

def patches2FullImage(patches):
    red = []
    green = []
    blue = []
    for patch_id in range(len(patches[0])): # for each patch
        patch = patches[:,patch_id]
        red_layer = np.reshape(patch[0:64],(8,8))
        green_layer = np.reshape(patch[64:128],(8,8))
        blue_layer = np.reshape(patch[128:192],(8,8))
        if len(red)==0:
            red = red_layer
            green = green_layer
            blue = blue_layer
        else:
            red = np.concatenate((red,red_layer),axis=1)
            green = np.concatenate((green,green_layer),axis=1)
            blue = np.concatenate((blue,blue_layer),axis=1)
    i = 0
    offset = i*32
    red_layer = np.concatenate((red[:,0+offset:32+offset],red[:,32+offset:64+offset],red[:,64+offset:96+offset],red[:,96+offset:128+offset]),axis=0)
    green_layer = np.concatenate((green[:,0 + offset:32], green[:,32 + offset:64], green[:,64 + offset:96], green[:,96 + offset:128]),axis=0)
    blue_layer = np.concatenate((blue[:,0 + offset:32], blue[:,32 + offset:64], blue[:,64 + offset:96], blue[:,96 + offset:128]),axis=0)

    red = []
    green = []
    blue = []
    for i in range(len(red_layer)):
        red = np.concatenate((red,red_layer[i,:]))
        green = np.concatenate((green, green_layer[i, :]))
        blue = np.concatenate((blue, blue_layer[i, :]))

    reconstruction = np.concatenate((red,green,blue))
    return reconstruction

def get_cifar_data():
    dict = unpickle(IMAGE_FILE)
    return dict[b'data']

def generate_sparse_codes():
    images = get_cifar_data()[0:100]
    sparse_codes = []
    for i in range(100):
        patches = fullImage2Patches(images[i])
        file = open("dictionary","rb")
        dict = np.load(file)
        file.close()
        sparse_codes.append(sparse_encode(patches.T,dict.T,algorithm="threshold"))

    file = open("sparse_codes","wb")
    np.save(file, sparse_codes)
    file.close()
generate_sparse_codes()



